In [ ]:
!pip install tensorflow scikit-learn pandas numpy matplotlib seaborn

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, LSTM, Dense, Dropout,
    Bidirectional, Conv1D, GlobalMaxPooling1D
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.19.0


In [3]:
from google.colab import files
uploaded = files.upload()

df = pd.read_csv(list(uploaded.keys())[0])

df.head()


Saving labeled_data.csv to labeled_data.csv


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [4]:
df = df[['tweet', 'class']]
df.dropna(inplace=True)

X = df['tweet']
y = df['class']

print("Unique Classes:", y.unique())


Unique Classes: [2 1 0]


In [5]:
tfidf = TfidfVectorizer(max_features=20000, stop_words='english')

X_tfidf = tfidf.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y, test_size=0.2, random_state=42
)


In [6]:
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

lr_pred = lr_model.predict(X_test)

lr_acc = accuracy_score(y_test, lr_pred)
print("Logistic Regression Accuracy:", lr_acc)
print(classification_report(y_test, lr_pred))


Logistic Regression Accuracy: 0.8928787573128909
              precision    recall  f1-score   support

           0       0.54      0.13      0.21       290
           1       0.91      0.97      0.94      3832
           2       0.84      0.82      0.83       835

    accuracy                           0.89      4957
   macro avg       0.76      0.64      0.66      4957
weighted avg       0.88      0.89      0.88      4957



In [7]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

nb_pred = nb_model.predict(X_test)

nb_acc = accuracy_score(y_test, nb_pred)
print("Naive Bayes Accuracy:", nb_acc)
print(classification_report(y_test, nb_pred))


Naive Bayes Accuracy: 0.8085535606213435
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       290
           1       0.80      1.00      0.89      3832
           2       0.94      0.22      0.36       835

    accuracy                           0.81      4957
   macro avg       0.58      0.41      0.42      4957
weighted avg       0.78      0.81      0.75      4957



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_train, y_train)

rf_pred = rf_model.predict(X_test)

rf_acc = accuracy_score(y_test, rf_pred)
print("Random Forest Accuracy:", rf_acc)
print(classification_report(y_test, rf_pred))


Random Forest Accuracy: 0.8880371192253379
              precision    recall  f1-score   support

           0       0.44      0.10      0.16       290
           1       0.90      0.97      0.93      3832
           2       0.85      0.80      0.82       835

    accuracy                           0.89      4957
   macro avg       0.73      0.62      0.64      4957
weighted avg       0.87      0.89      0.87      4957



In [9]:
MAX_WORDS = 20000
MAX_LEN = 200

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X)

sequences = tokenizer.texts_to_sequences(X)
padded_sequences = pad_sequences(sequences, maxlen=MAX_LEN, padding='post')

X_train_dl, X_test_dl, y_train_dl, y_test_dl = train_test_split(
    padded_sequences, y, test_size=0.2, random_state=42
)


In [11]:
lstm_model = Sequential([
    Embedding(MAX_WORDS, 128, input_length=MAX_LEN),
    LSTM(64),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')   # 3 classes
])

lstm_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

lstm_model.summary()

lstm_model.fit(
    X_train_dl, y_train_dl,
    epochs=1,
    batch_size=64,
    validation_split=0.2
)

lstm_loss, lstm_acc = lstm_model.evaluate(X_test_dl, y_test_dl)
print("LSTM Accuracy:", lstm_acc)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

248/248 ━━━━━━━━━━━━━━━━━━━━ 77s 300ms/step - accuracy: 0.7569 - loss: 0.7136 - val_accuracy: 0.7844 - val_loss: 0.6415
155/155 ━━━━━━━━━━━━━━━━━━━━ 7s 47ms/step - accuracy: 0.7743 - loss: 0.6664
LSTM Accuracy: 0.7730482220649719


In [12]:
bilstm_model = Sequential([
    Embedding(MAX_WORDS, 128, input_length=MAX_LEN),
    Bidirectional(LSTM(64)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

bilstm_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

bilstm_model.fit(
    X_train_dl, y_train_dl,
    epochs=1,
    batch_size=64,
    validation_split=0.2
)

bilstm_loss, bilstm_acc = bilstm_model.evaluate(X_test_dl, y_test_dl)
print("BiLSTM Accuracy:", bilstm_acc)


248/248 ━━━━━━━━━━━━━━━━━━━━ 144s 548ms/step - accuracy: 0.7888 - loss: 0.5931 - val_accuracy: 0.9034 - val_loss: 0.3037
155/155 ━━━━━━━━━━━━━━━━━━━━ 9s 61ms/step - accuracy: 0.8867 - loss: 0.3312
BiLSTM Accuracy: 0.8862215280532837


In [13]:
cnn_model = Sequential([
    Embedding(MAX_WORDS, 128, input_length=MAX_LEN),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

cnn_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

cnn_model.fit(
    X_train_dl, y_train_dl,
    epochs=1,
    batch_size=64,
    validation_split=0.2
)

cnn_loss, cnn_acc = cnn_model.evaluate(X_test_dl, y_test_dl)
print("CNN Accuracy:", cnn_acc)


248/248 ━━━━━━━━━━━━━━━━━━━━ 48s 188ms/step - accuracy: 0.7859 - loss: 0.5915 - val_accuracy: 0.9183 - val_loss: 0.2586
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.9025 - loss: 0.2836
CNN Accuracy: 0.9043776392936707


In [14]:
results = pd.DataFrame({
    "Model": [
        "Logistic Regression",
        "Naive Bayes",
        "Random Forest",
        "LSTM",
        "BiLSTM",
        "CNN"
    ],
    "Accuracy": [
        lr_acc,
        nb_acc,
        rf_acc,
        lstm_acc,
        bilstm_acc,
        cnn_acc
    ]
})

results


,Model,Accuracy
0,Logistic Regression,0.892879
1,Naive Bayes,0.808554
2,Random Forest,0.888037
3,LSTM,0.773048
4,BiLSTM,0.886222
5,CNN,0.904378


In [15]:
!pip install mistralai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.0/503.0 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 13.8 MB/s eta 0:00:00


In [17]:
from mistralai import Mistral

api_key = "YOUR_MISTRAL_API_KEY"
client = Mistral(api_key='C9B2bQOizZU6XXdW8bIONOYaZZ4brXNZ')

model_name = "mistral-large-latest"  # or mistral-small-latest

def ask_mistral(prompt):
    response = client.chat.complete(
        model=model_name,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()


In [18]:
def zero_shot(tweet):
    prompt = f"""
    Classify this tweet into:
    0 = Hate Speech
    1 = Offensive Language
    2 = Neither

    Tweet: "{tweet}"
    """
    return ask_mistral(prompt)


def one_shot(tweet):
    prompt = f"""
    Example:
    Tweet: "I hate all of you people."
    Class: 0

    Now classify:
    Tweet: "{tweet}"
    """
    return ask_mistral(prompt)


def few_shot(tweet):
    prompt = f"""
    Examples:

    Tweet: "I hate you."
    Class: 0

    Tweet: "You are stupid."
    Class: 1

    Tweet: "Have a wonderful day!"
    Class: 2

    Now classify:
    Tweet: "{tweet}"
    """
    return ask_mistral(prompt)


def chain_of_thought(tweet):
    prompt = f"""
    Classify the tweet into:
    0 = Hate Speech
    1 = Offensive Language
    2 = Neither

    Think step-by-step:
    - Does it target a protected group?
    - Is it abusive?
    - Is it neutral?

    Tweet: "{tweet}"

    Explain your reasoning and then give final class number.
    """
    return ask_mistral(prompt)




In [19]:
sample_df = df.sample(5, random_state=42)

for index, row in sample_df.iterrows():
    tweet = row['tweet']
    true_label = row['class']

    print("="*80)
    print("TWEET:")
    print(tweet)
    print("True Label:", true_label)
    print("\n--- ZERO SHOT RESPONSE ---")
    print(zero_shot(tweet))

    print("\n--- ONE SHOT RESPONSE ---")
    print(one_shot(tweet))

    print("\n--- FEW SHOT RESPONSE ---")
    print(few_shot(tweet))

    print("\n--- CHAIN OF THOUGHT RESPONSE ---")
    print(chain_of_thought(tweet))

    print("="*80)
    print("\n\n")


TWEET:
934 8616
i got a missed call from yo bitch
True Label: 1

--- ZERO SHOT RESPONSE ---
The tweet contains offensive language ("yo bitch") but does not appear to target a specific group with hate or promote violence, discrimination, or hostility based on protected characteristics (e.g., race, religion, gender, etc.).

**Classification: 1 = Offensive Language**

--- ONE SHOT RESPONSE ---
Class: **0** (Non-Hate Speech)

**Reasoning:**
The tweet contains informal language and a crude reference ("yo bitch"), but it does not explicitly target a group based on protected characteristics (e.g., race, religion, gender, etc.) or promote violence/harm. The tone is more confrontational or vulgar rather than hateful. Thus, it does not meet the criteria for hate speech (Class 1).

--- FEW SHOT RESPONSE ---
Based on the provided examples, the classification appears to be:

- **Class 0**: Hate speech or strong negative sentiment (e.g., "I hate you.")
- **Class 1**: Insults or offensive language (e